In [ ]:
pip install transformers datasets sentencepiece

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 325 kB 45.8 MB/s 
     |████████████████████████████████| 1.2 MB 41.9 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 32.7 MB/s 
     |████████████████████████████████| 596 kB 44.6 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
     |████████████████████████████████| 1.1 MB 34.0 MB/s 
     |████████████████████████████████| 212 kB 32.8 MB/s 
     |████████████████████████████████| 134 kB 34.3 MB/s 
     |████████████████████████████████| 127 kB 17.8 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 46.7 MB/s 
     |████████████████████████████████| 271 kB 46.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
model_name = "aubmindlab/bert-base-arabertv02"

tokenizer = BertTokenizer.from_pretrained(model_name)
model_path= "/content/drive/MyDrive/Quran_QA/quranBERT/checkpoint-6236"
model = BertForMaskedLM.from_pretrained(model_path)

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

In [ ]:
# tokenizer.save_pretrained("arabert")

('arabert/tokenizer_config.json',
 'arabert/special_tokens_map.json',
 'arabert/vocab.txt',
 'arabert/added_tokens.json')

In [ ]:
quranpath = '/content/drive/MyDrive/Quran_QA/trials/Embedding_corpus/quran.txt'
with open(quranpath , 'r' , encoding= 'utf-8') as f:
    lines = f.readlines()

In [ ]:
for i in range(len(lines)):
  lines[i] = lines[i][:-1]

In [ ]:
import numpy as np
import heapq
verse_lengths = [len(x.split()) for x in lines]
maxlength =  max(verse_lengths)
maxlengthid =  np.argmax([verse_lengths])
verse_lengths = sorted([(i , x) for i , x in enumerate(verse_lengths)] ,key =lambda x : x[1] )
maxlength

129

In [ ]:
verse_lengths[-6:]

[(446, 75), (2851, 76), (2821, 78), (5494, 78), (504, 88), (288, 129)]

In [ ]:
lines[5494]

'إن ربك يعلم أنك تقوم أدنى من ثلثي الليل ونصفه وثلثه وطائفة من الذين معك  والله يقدر الليل والنهار  علم أن لن تحصوه فتاب عليكم  فاقرءوا ما تيسر من القرآن  علم أن سيكون منكم مرضى  وآخرون يضربون في الأرض يبتغون من فضل الله  وآخرون يقاتلون في سبيل الله  فاقرءوا ما تيسر منه  وأقيموا الصلاة وآتوا الزكاة وأقرضوا الله قرضا حسنا  وما تقدموا لأنفسكم من خير تجدوه عند الله هو خيرا وأعظم أجرا  واستغفروا الله  إن الله غفور رحيم'

In [ ]:
from datasets import load_dataset
dataset = load_dataset('text', data_files={'train': [quranpath], 'test':quranpath})

Using custom data configuration default-4e1b064d85e22c26


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-4e1b064d85e22c26/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6236
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6236
    })
})

In [ ]:

def tokenize_pad_and_truncate(texts):
    return tokenizer(texts["text"], padding="max_length")

tokenized_datasets = dataset.map(tokenize_pad_and_truncate, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import  TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Quran_QA/quranBERT_moreepochs",
    save_strategy='epoch',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=4,
    weight_decay=0.01,
)

In [ ]:
from random import randint
testset =tokenized_datasets['test'].select([randint(0 , 6000) for _ in range(200)])
testset

Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets['train'] ,
    eval_dataset = testset,
    data_collator=data_collator,
    tokenizer= tokenizer
)

In [ ]:
len(tokenized_datasets['train'])

6236

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6236
  Num Epochs = 4
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 12472


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.964900,1.282374
2,0.851300,1.062868
3,0.911500,1.053108
4,1.122000,0.896188


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 2
Saving model checkpoint to /content/drive/MyDrive/Quran_QA/quranBERT_moreepochs/checkpoint-3118
Configuration saved in /content/drive/MyDrive/Quran_QA/quranBERT_moreepochs/checkpoint-3118/config.json
Model weights saved in /content/drive/MyDrive/Quran_QA/quranBERT_moreepochs/checkpoint-3118/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Quran_QA/quranBERT_moreepochs/checkpoint-3118/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Quran_QA/quranBERT_moreepochs/checkpoint-3118/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text

TrainOutput(global_step=12472, training_loss=0.9797804014586423, metrics={'train_runtime': 8227.6548, 'train_samples_per_second': 3.032, 'train_steps_per_second': 1.516, 'total_flos': 6567946201595904.0, 'train_loss': 0.9797804014586423, 'epoch': 4.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Quran_QA/quranBERT/8epochs")

Configuration saved in /content/drive/MyDrive/Quran_QA/quranBERT/8epochs/config.json
Model weights saved in /content/drive/MyDrive/Quran_QA/quranBERT/8epochs/pytorch_model.bin


In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Quran_QA/quranBERT/8epochs")


tokenizer config file saved in /content/drive/MyDrive/Quran_QA/quranBERT/8epochs/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Quran_QA/quranBERT/8epochs/special_tokens_map.json


('/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/tokenizer_config.json',
 '/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/special_tokens_map.json',
 '/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/vocab.txt',
 '/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/added_tokens.json')

In [ ]:
sample = tokenizer([" اياك نعبد واياك  [MASK]"] , return_tensors='pt').to("cuda")
result =model(**sample)
result

MaskedLMOutput([('logits',
                 tensor([[[ -4.3877,   4.1595, -16.1236,  ...,  -6.3789,  -6.3800,  -6.3765],
                          [ -1.2115,  16.2258,  -6.8807,  ...,  -7.1434,  -7.1438,  -7.1422],
                          [ -6.3657,  15.5120,   0.9941,  ...,  -8.9898,  -8.9895,  -8.9913],
                          ...,
                          [ -1.8498,   7.8121,  -8.0505,  ...,  -7.6740,  -7.6748,  -7.6721],
                          [ -4.8059,  15.2867, -12.3890,  ...,  -5.1282,  -5.1301,  -5.1229],
                          [ -6.1410,   1.8261, -15.1888,  ...,  -5.7298,  -5.7316,  -5.7247]]],
                        device='cuda:0', grad_fn=<AddBackward0>))])

In [ ]:
import numpy as np

ids =np.argmax(result[0].detach().cpu().numpy(), axis= 2)
print(ids)
tokenizer.decode(ids[0])

[[ 5431 20833   209  2843   400  7363  1482 16915  6388]]


'الضر اياك نعبد واياك المصيرتك'